In [1]:
import pandas as pd
import os
import math


In [2]:
#functions to save data frame
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()  

In [3]:
def calcualteEs(F,gamma):
    elasticity=30.0*(1-0.5*F)*math.pow(gamma,9.0)
    elasticity=float(math.ceil(elasticity))
    return elasticity
def calculateHeff(B,F,gamma,sigmaNot):
    term1=gamma*B+1.5*F*math.pow(gamma,2.5)
    term2=8*sigmaNot
    term3=math.log(term1/term2)
    effective_height=1.4*B*(1-3.8*gamma-term3)
    effective_height=round(effective_height,2)
    return  effective_height

def calcualteLiftSettlement(Heff,B,sigmaNot,Es):
    A=0.555*math.log(Heff/B)+0.93
    delh=(A*B*sigmaNot)/Es
    return delh
def calcualteContactWidth(gamma_target,Roller_Diameter):
    B=(0.45-0.15*gamma_target)*Roller_Diameter
    B=math.floor(B*100)
    return B
def calculateRequiredMinimumSigma(F,gamma_target,B):
    term1=(F*math.pow(gamma_target,2.5))/4.0
    term2=(gamma_target*B)/6.0
    term3=math.exp(3.8*gamma_target-1)
    sigma_min=(term1+term2)*term3
    return sigma_min
def calculateMaximumLiftThicknessFromSigma(sigma,F,gamma_target):
    term1=math.exp(3.8*gamma_target-1)
    term2=sigma/term1
    term3=(F*math.pow(gamma_target,2.5))/4.0
    term4=term2-term3
    B=(term4*6)/gamma_target
    return B
    
def calcualteMinimumComaptorWeight(F,gamma_target,Roller_diameter):
    B=calcualteContactWidth(gamma_target,Roller_diameter)
    B=B/100.0
    term1=(F*math.pow(gamma_target,2.5))/4.0
    term2=(gamma_target*B)/6.0
    term3=math.exp(3.8*gamma_target-1)
    sigma_min=(term1+term2)*term3
    I=1.0
    S=0.5
    minimum_compactor_weight=(sigma_min*S)/(1.2*I)
    minimum_compactor_weight=math.ceil(minimum_compactor_weight)
    sigma_min2=math.floor(sigma_min)
    myvalue={"mcw":minimum_compactor_weight,"sigma":sigma_min2}
    return myvalue
def calcualteRollerPasses(gamma_initial,gamma_final,F,Roller_diameter):
    B=calcualteContactWidth(gamma_final,Roller_diameter)
    myvalue=calcualteMinimumComaptorWeight(F,gamma_final,Roller_diameter)
    sigma=myvalue["sigma"]
    t1=17.5*(1-0.5*F)
    t2=t1/sigma
    t3=math.pow(gamma_final,7)-math.pow(gamma_initial,7)
    no_passes=math.ceil(t2*t3)
    myreturn_val={"LT":B,"CWT":myvalue["mcw"],"Sigma":myvalue["sigma"],"passes":no_passes}    
    return myreturn_val
def calcualteRollerPassesForFixedLiftThickness(gamma_initial,gamma_final,F,Roller_Weight,Roller_Dia,Lift_thicness):
    #B=calcualteContactWidth(gamma_final,Roller_diameter)
    B=Lift_thicness/100.0  
    B_Max=calcualteContactWidth(gamma_final,Roller_Dia)/100.00    
    if B>B_Max:
        B=B_Max      
    #myvalue=calcualteMinimumComaptorWeight(F,gamma_final,Roller_diameter)
    I=1.0
    S=0.5
    sigma=(Roller_Weight*1.2*I)/S
    sigma_min=calculateRequiredMinimumSigma(F,gamma_final,B)
    if sigma_min>sigma:
        B_revised=calculateMaximumLiftThicknessFromSigma(sigma,F,gamma_final)
        print("Lift Thicknes Revision necessary revised LT={} cm".format(B_revised*100))
        
    t1=17.5*(1-0.5*F)
    t2=t1/sigma
    t3=math.pow(gamma_final,7)-math.pow(gamma_initial,7)
    no_passes=math.ceil(t2*t3)
    myreturn_val={"LT":B,"Sigma":sigma,"passes":no_passes}    
    return myreturn_val  

    

In [4]:
myfolder=r'F:\website\cmis6\Civilworks cost\Soil Compaction' # office computer
input_path=os.path.join(myfolder,"Input.xlsx")
output_path=os.path.join(myfolder,"Output.xlsx")
sheetName="Parameter"
parameter_df=pd.read_excel(input_path,sheet_name=sheetName)
parameter_df
"""Extracting Design Parameter"""
gammaInitial=parameter_df.iloc[0,2]
liftThicknessInitial=parameter_df.iloc[1,2]
sigmaNot=parameter_df.iloc[2,2]
fine_percent=parameter_df.iloc[3,2]
contact_width=parameter_df.iloc[4,2]
Hn=liftThicknessInitial
gamma_old=gammaInitial
for i in range(1,9):
    print("gamma_ini={} Hi={} sigmaNot={}".format(gamma_old,Hn,sigmaNot))
es=calcualteEs(fine_percent,gammaInitial)
Heff= calculateHeff(contact_width,fine_percent,gammaInitial,sigmaNot)
settlement=calcualteLiftSettlement(Heff,contact_width,sigmaNot,es)

Hn1=Hn-settlement
gamma_new=round((Hn/Hn1)*gamma_old,2)
print("Youn's Modulus={} Heff={} settlement={} gamma_new={}".format(es,Heff,settlement,gamma_new))
#F,gamma_target,B
myval=calcualteMinimumComaptorWeight(0.10,1.75,1.2)
B=calcualteContactWidth(1.75,1.2)
print("Comapctor Weight={} metric_ton compaction_stress={} psi lift_thickness={} cm".format(myval["mcw"],myval["sigma"],B))
myvalue=calcualteRollerPasses(1.70,1.75,0.50,1.2)
print("Roller={}mton sigma={} lift={} cm passe={} nos".format(myvalue["CWT"],myvalue["Sigma"],myvalue["LT"],myvalue["passes"]))
myvalue=calcualteRollerPassesForFixedLiftThickness(1.39,1.75,0.66,20,1.2,22)
print("sigma={} Thickness={} no of passes={}".format(myvalue["Sigma"],myvalue["LT"],myvalue["passes"]))

gamma_ini=1.4 Hi=2.0 sigmaNot=41.4
gamma_ini=1.4 Hi=2.0 sigmaNot=41.4
gamma_ini=1.4 Hi=2.0 sigmaNot=41.4
gamma_ini=1.4 Hi=2.0 sigmaNot=41.4
gamma_ini=1.4 Hi=2.0 sigmaNot=41.4
gamma_ini=1.4 Hi=2.0 sigmaNot=41.4
gamma_ini=1.4 Hi=2.0 sigmaNot=41.4
gamma_ini=1.4 Hi=2.0 sigmaNot=41.4
Youn's Modulus=443.0 Heff=0.29 settlement=0.027250977402278876 gamma_new=1.42
Comapctor Weight=20 metric_ton compaction_stress=47 psi lift_thickness=22 cm
Roller=68mton sigma=162 lift=22 cm passe=1 nos
Lift Thicknes Revision necessary revised LT=-171.29993125384482 cm
sigma=48.0 Thickness=0.22 no of passes=10


In [5]:
input_path

'F:\\website\\cmis6\\Civilworks cost\\Soil Compaction\\Input.xlsx'

In [6]:
myframes=[]
mynames=[]
myframes.append(parameter_df)
mynames.append("Input Parameter")
saveDataFrame(myframes,output_path,mynames)

In [7]:
math.log(10)

2.302585092994046